In [1]:
import tensorflow_datasets as tfds
ds_train_tf, ds_validation_tf, ds_test_tf, ds_prediction_tf = tfds.load(
    name='titanic',
    split=['train[:70%]', 'train[70%:80%]', 'train[80%:90%]', 'train[90%:]']
)

2022-11-24 13:53:56.741469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 13:53:56.852331: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-24 13:53:56.855486: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 13:53:56.855505: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
for x in ds_train_tf.take(1):
    print(f"x={x}")

x={'age': <tf.Tensor: shape=(), dtype=float32, numpy=30.0>, 'boat': <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>, 'body': <tf.Tensor: shape=(), dtype=int32, numpy=-1>, 'cabin': <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>, 'embarked': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'fare': <tf.Tensor: shape=(), dtype=float32, numpy=13.0>, 'home.dest': <tf.Tensor: shape=(), dtype=string, numpy=b'Sarnia, ON'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'McCrie, Mr. James Matthew'>, 'parch': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'pclass': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'sex': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'sibsp': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'survived': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'ticket': <tf.Tensor: shape=(), dtype=string, numpy=b'233478'>}


2022-11-24 13:53:59.247488: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [3]:
import pandas as pd

In [4]:
ds_train = tfds.as_dataframe(ds_train_tf)
ds_val = tfds.as_dataframe(ds_validation_tf)
ds_test = tfds.as_dataframe(ds_test_tf)

y_train = ds_train['survived']
y_val = ds_val['survived']
y_test = ds_test['survived']

ds_train = ds_train.drop(['survived'], axis=1)
ds_test = ds_test.drop(['survived'], axis=1)
ds_val = ds_val.drop(['survived'], axis=1)

In [5]:
ds_train

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,ticket
0,30.0,b'Unknown',-1,b'Unknown',2,13.000000,"b'Sarnia, ON'","b'McCrie, Mr. James Matthew'",0,1,0,0,b'233478'
1,37.0,b'Unknown',98,b'Unknown',2,7.925000,"b'Ruotsinphytaa, Finland New York, NY'","b'Gustafsson, Mr. Anders Vilhelm'",0,2,0,2,b'3101276'
2,28.0,b'9',-1,b'Unknown',2,13.000000,b'Spain',"b'Reynaldo, Ms. Encarnacion'",0,1,1,0,b'230434'
3,18.0,b'Unknown',-1,b'Unknown',2,73.500000,"b'Lyndhurst, England'","b'Davies, Mr. Charles Henry'",0,1,0,0,b'S.O.C. 14879'
4,-1.0,b'Unknown',-1,b'Unknown',0,7.895800,b'Unknown',"b'Gheorgheff, Mr. Stanio'",0,2,0,0,b'349254'
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,21.0,b'12',-1,b'Unknown',2,10.500000,"b'Guernsey / Wilmington, DE'","b'Rugg, Miss. Emily'",0,1,1,0,b'C.A. 31026'
912,36.0,b'Unknown',-1,b'Unknown',2,7.895800,b'Unknown',"b'Turcin, Mr. Stjepan'",0,2,0,0,b'349247'
913,-1.0,b'Unknown',-1,b'Unknown',2,7.550000,b'Unknown',"b'MacKay, Mr. George William'",0,2,0,0,b'C.A. 42795'
914,39.0,b'2',-1,b'Unknown',2,211.337494,b'Unknown',"b'Kreuchen, Miss. Emilie'",0,0,1,0,b'24160'


In [6]:
ds_train.describe()

,age,body,embarked,fare,parch,pclass,sex,sibsp
count,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000
mean,23.807041,15.080786,1.485808,34.101959,0.395197,1.292576,0.346070,0.496725
std,18.128685,58.199976,0.822375,54.664959,0.878983,0.841278,0.475976,1.059431
min,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,-1.000000,1.000000,7.895800,0.000000,0.750000,0.000000,0.000000
50%,24.750000,-1.000000,2.000000,14.454200,0.000000,2.000000,0.000000,0.000000
75%,36.000000,-1.000000,2.000000,31.303125,0.000000,2.000000,1.000000,1.000000
max,80.000000,327.000000,3.000000,512.329224,9.000000,2.000000,1.000000,8.000000


In [7]:
ds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        916 non-null    float32
 1   boat       916 non-null    object 
 2   body       916 non-null    int32  
 3   cabin      916 non-null    object 
 4   embarked   916 non-null    int64  
 5   fare       916 non-null    float32
 6   home.dest  916 non-null    object 
 7   name       916 non-null    object 
 8   parch      916 non-null    int32  
 9   pclass     916 non-null    int64  
 10  sex        916 non-null    int64  
 11  sibsp      916 non-null    int32  
 12  ticket     916 non-null    object 
dtypes: float32(2), int32(3), int64(3), object(5)
memory usage: 75.3+ KB


In [8]:
num_features = ['age', 'fare']
# Не берем только body и name
cat_features = ['boat', 'cabin', 'embarked', 'home.dest', 'parch', 'pclass', 'sex',	'sibsp', 'ticket']

In [9]:
import category_encoders as ce
te = ce.target_encoder.TargetEncoder(cols=cat_features)
ds_train = te.fit_transform(ds_train, y_train)
ds_val = te.transform(ds_val)
ds_test = te.transform(ds_test)

/usr/local/lib/python3.10/dist-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/usr/local/lib/python3.10/dist-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [10]:
features = num_features + cat_features

In [11]:
ds_train[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,30.0,13.000000,0.022147,0.288326,0.318612,0.365721,0.319484,0.410526,0.191987,0.329582,0.365721
1,37.0,7.925000,0.022147,0.288326,0.318612,0.098357,0.319484,0.235412,0.191987,0.428571,0.365721
2,28.0,13.000000,0.999999,0.288326,0.318612,0.365721,0.319484,0.410526,0.694006,0.329582,0.365721
3,18.0,73.500000,0.022147,0.288326,0.318612,0.365721,0.319484,0.410526,0.191987,0.329582,0.000904
4,-1.0,7.895800,0.022147,0.288326,0.523316,0.253695,0.319484,0.235412,0.191987,0.329582,0.365721
...,...,...,...,...,...,...,...,...,...,...,...
911,21.0,10.500000,0.999996,0.288326,0.318612,0.365721,0.319484,0.410526,0.694006,0.329582,0.365721
912,36.0,7.895800,0.022147,0.288326,0.318612,0.253695,0.319484,0.235412,0.191987,0.329582,0.365721
913,-1.0,7.550000,0.022147,0.288326,0.318612,0.253695,0.319484,0.235412,0.191987,0.329582,0.365721
914,39.0,211.337494,0.999787,0.288326,0.318612,0.253695,0.319484,0.611354,0.694006,0.329582,0.829416


In [12]:
from sklearn.preprocessing import StandardScaler


ss = StandardScaler()
ds_train[features] = ss.fit_transform(ds_train[features])
ds_val[features] = ss.transform(ds_val[features])
ds_test[features] = ss.transform(ds_test[features])

In [13]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
input_shape = (len(features),)
input_shape

(11,)

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses

In [16]:
model = keras.Sequential()

model.add(layers.InputLayer(input_shape = input_shape,
                            ))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es_callback = EarlyStopping(patience=3)
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_f1_m', verbose=1, save_best_only=True, mode='max')

In [18]:
model.compile(
    optimizer = optimizers.Adam(learning_rate = 0.01),
    loss = 'BCE',
    metrics = [f1_m, precision_m, recall_m],

)


history = model.fit(ds_train[features],
                    y_train, 
                    epochs=30, batch_size=64, callbacks=[checkpoint, es_callback],
                    validation_data = (ds_val[features], y_val))

Epoch 1/30
 1/15 [=>............................] - ETA: 12s - loss: 0.6749 - f1_m: 0.4545 - precision_m: 0.4762 - recall_m: 0.4348
Epoch 1: val_f1_m improved from -inf to 0.94189, saving model to model.h5
15/15 [==============================] - 1s 18ms/step - loss: 0.2193 - f1_m: 0.8962 - precision_m: 0.9229 - recall_m: 0.8818 - val_loss: 0.3796 - val_f1_m: 0.9419 - val_precision_m: 0.9615 - val_recall_m: 0.9286
Epoch 2/30
 1/15 [=>............................] - ETA: 0s - loss: 0.0430 - f1_m: 0.9767 - precision_m: 0.9545 - recall_m: 1.0000
Epoch 2: val_f1_m improved from 0.94189 to 0.94815, saving model to model.h5
15/15 [==============================] - 0s 4ms/step - loss: 0.0703 - f1_m: 0.9660 - precision_m: 0.9799 - recall_m: 0.9551 - val_loss: 0.2185 - val_f1_m: 0.9481 - val_precision_m: 0.9733 - val_recall_m: 0.9286
Epoch 3/30
 1/15 [=>............................] - ETA: 0s - loss: 0.0563 - f1_m: 0.9811 - precision_m: 1.0000 - recall_m: 0.9630
Epoch 3: val_f1_m did not improv

In [19]:
from keras.models import load_model
model = load_model('model.h5', custom_objects={'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})

In [20]:
ds_train[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,0.341798,-0.386234,-0.742014,-0.341077,-0.547764,0.071825,-0.473613,0.285015,-0.727472,-0.382689,-0.163563
1,0.728137,-0.479123,-0.742014,-0.341077,-0.547764,-1.303872,-0.473613,-0.828906,-0.727472,0.661350,-0.163563
2,0.231415,-0.386234,1.398089,-0.341077,-0.547764,0.071825,-0.473613,0.285015,1.374624,-0.382689,-0.163563
3,-0.320498,0.721112,-0.742014,-0.341077,-0.547764,0.071825,-0.473613,0.285015,-0.727472,-0.382689,-1.885922
4,-1.369134,-0.479658,-0.742014,-0.341077,1.851442,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
...,...,...,...,...,...,...,...,...,...,...,...
911,-0.154924,-0.431993,1.398084,-0.341077,-0.547764,0.071825,-0.473613,0.285015,1.374624,-0.382689,-0.163563
912,0.672946,-0.479658,-0.742014,-0.341077,-0.547764,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
913,-1.369134,-0.485987,-0.742014,-0.341077,-0.547764,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
914,0.838520,3.243986,1.397627,-0.341077,-0.547764,-0.504596,-0.473613,1.562503,1.374624,-0.382689,2.025621


In [21]:
from sklearn.metrics import f1_score

In [22]:
predict = model.predict(ds_test[features])

f1_score(y_test, predict>=0.5)

5/5 [==============================] - 0s 1ms/step


0.975609756097561

In [23]:
ds_prediction = tfds.as_dataframe(ds_prediction_tf)
ds_prediction = ds_prediction.drop(['survived'], axis=1)

In [24]:
ds_prediction = te.transform(ds_prediction)
ds_prediction[features] = ss.transform(ds_prediction[features])

In [25]:
ds_prediction[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,-1.267951,-0.093382,1.398093,-0.341077,-0.547764,0.071825,0.859443,0.285015,-0.727472,-0.382689,-0.163563
1,-1.369134,-0.198626,1.398091,-0.341077,-0.240507,-0.504596,-0.473613,-0.828906,1.374624,0.661350,-0.163563
2,-0.982794,1.837610,1.398093,0.236351,1.851442,0.071825,0.859443,1.562503,-0.727472,-0.382689,-0.163563
3,0.286606,-0.450601,-0.742014,-0.341077,-0.547764,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
4,-1.369134,-0.624177,-0.742014,0.236351,-0.547764,-0.504596,-0.473613,1.562503,-0.727472,-0.382689,-0.163563
...,...,...,...,...,...,...,...,...,...,...,...
126,0.452180,-0.482326,-0.742014,-0.341077,-0.240507,-1.585648,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
127,-0.265307,-0.481869,-0.742014,-0.341077,-0.547764,0.071825,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
128,-1.369134,-0.475234,-0.742014,-0.341077,-0.240507,-0.504596,-0.473613,-0.828906,1.374624,-0.382689,-0.163563
129,0.396989,-0.248502,0.009924,-0.341077,-0.547764,0.576932,2.245934,-0.828906,1.374624,1.437733,0.299896


In [26]:
import numpy as np

predict_platform = model.predict(ds_prediction[features])
predictions = np.array([1 if predict_platform[_] >= 0.5 else 0 for _ in range(len(predict_platform))])
predictions

5/5 [==============================] - 0s 1ms/step


array([1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1])

In [27]:
ds_prediction[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,-1.267951,-0.093382,1.398093,-0.341077,-0.547764,0.071825,0.859443,0.285015,-0.727472,-0.382689,-0.163563
1,-1.369134,-0.198626,1.398091,-0.341077,-0.240507,-0.504596,-0.473613,-0.828906,1.374624,0.661350,-0.163563
2,-0.982794,1.837610,1.398093,0.236351,1.851442,0.071825,0.859443,1.562503,-0.727472,-0.382689,-0.163563
3,0.286606,-0.450601,-0.742014,-0.341077,-0.547764,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
4,-1.369134,-0.624177,-0.742014,0.236351,-0.547764,-0.504596,-0.473613,1.562503,-0.727472,-0.382689,-0.163563
...,...,...,...,...,...,...,...,...,...,...,...
126,0.452180,-0.482326,-0.742014,-0.341077,-0.240507,-1.585648,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
127,-0.265307,-0.481869,-0.742014,-0.341077,-0.547764,0.071825,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
128,-1.369134,-0.475234,-0.742014,-0.341077,-0.240507,-0.504596,-0.473613,-0.828906,1.374624,-0.382689,-0.163563
129,0.396989,-0.248502,0.009924,-0.341077,-0.547764,0.576932,2.245934,-0.828906,1.374624,1.437733,0.299896


In [28]:
import pandas as pd
predict_task = pd.read_csv('DL_Task_3_Titain_reserved.csv')
predict_task

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,ticket
0,0.8333,b'13',-1,b'Unknown',2,29.0000,"b'Bangkok, Thailand / Roseville, IL'","b'Caldwell, Master. Alden Gates'",2,1,0,0,b'248738'
1,-1.0000,b'16',-1,b'Unknown',1,23.2500,b'Unknown',"b'McCoy, Miss. Alicia'",0,2,1,2,b'367226'
2,6.0000,b'3',-1,b'E34',0,134.5000,"b'Tuxedo Park, NY'","b'Spedden, Master. Robert Douglas'",2,0,0,0,b'16966'
3,29.0000,b'Unknown',-1,b'Unknown',2,9.4833,b'Unknown',"b'Larsson, Mr. August Viktor'",0,2,0,0,b'7545'
4,-1.0000,b'Unknown',-1,b'B102',2,0.0000,b'Unknown',"b'Fry, Mr. Richard'",0,0,0,0,b'112058'
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,32.0000,b'Unknown',-1,b'Unknown',1,7.7500,"b'Ireland New York, NY'","b'Dooley, Mr. Patrick'",0,2,0,0,b'370376'
127,19.0000,b'Unknown',-1,b'Unknown',2,7.7750,"b'Myren, Sweden New York, NY'","b'Gustafsson, Mr. Karl Gideon'",0,2,0,0,b'347069'
128,-1.0000,b'Unknown',-1,b'Unknown',1,8.1375,b'Unknown',"b'Peters, Miss. Katie'",0,2,1,0,b'330935'
129,31.0000,b'C D',-1,b'Unknown',2,20.5250,"b'Strood, Kent, England Detroit, MI'","b'Goldsmith, Mrs. Frank John (Emily Alice Brown)'",1,2,1,1,b'363291'


In [29]:
predict_task = te.transform(predict_task)
predict_task[features] = ss.transform(predict_task[features])

In [30]:
predict_task[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,-1.267951,-0.093382,0.009924,0.236351,-0.547764,0.071825,0.859443,0.285015,-0.727472,-0.382689,-0.163563
1,-1.369134,-0.198626,0.009924,0.236351,-0.240507,0.071825,-0.473613,-0.828906,1.374624,0.661350,-0.163563
2,-0.982794,1.837610,0.009924,0.236351,1.851442,0.071825,0.859443,1.562503,-0.727472,-0.382689,-0.163563
3,0.286606,-0.450601,0.009924,0.236351,-0.547764,0.071825,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
4,-1.369134,-0.624177,0.009924,0.236351,-0.547764,0.071825,-0.473613,1.562503,-0.727472,-0.382689,-0.163563
...,...,...,...,...,...,...,...,...,...,...,...
126,0.452180,-0.482326,0.009924,0.236351,-0.240507,0.071825,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
127,-0.265307,-0.481869,0.009924,0.236351,-0.547764,0.071825,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
128,-1.369134,-0.475234,0.009924,0.236351,-0.240507,0.071825,-0.473613,-0.828906,1.374624,-0.382689,-0.163563
129,0.396989,-0.248502,0.009924,0.236351,-0.547764,0.071825,2.245934,-0.828906,1.374624,1.437733,-0.163563


In [31]:
predict_platform1 = model.predict(predict_task[features])
predictions1 = np.array([1 if predict_platform1[_] >= 0.5 else 0 for _ in range(len(predict_platform1))])
predictions1

5/5 [==============================] - 0s 957us/step


array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0])